# Investigating Mine Blasting Data Using Clustering Techniques
<strong>Project Plan v1.0</strong><br/>
Date: May 1, 2017<br/>
Version: V4<br/>
Author: Amol Rao<br/>
Student Number: 994610453<br/>
Course: MIE1512

## Contents 
The contents of this notebook are as follows:

### 1. Introduction

### 2. Project Plan & Task Breakdown

### 3. Data Understanding

### 4. Data Preparation

### 5. Modeling 

### 6. Alternative Iterations

### 7. Conclusions & Recommendations

### 8. Bibliography

## 1.0) Introduction

### 1.1) Motivation of the Problem

The motivation for this investigation is to better understand the factors driving the relationship between blast outcomes and rock mass parameters for an open pit gold mine located in North Eastern Ontario. 

The blasting of rock at mines serves two primary purposes: a) to enable access to the mineral deposit containing the ore and b) fragment the ore to a suitable size for digging and processing. 

Both objectives require optimization of the blasting process to minimize cost and maximize speed at which fragmented rock can be moved on for processing through improved digability. The efficiency of the blasting operation depends on a few key parameters, including; rock characteristics, properties and quantities of explosives, blast geometry, blast size, priming method and initiation sequence. 

As part of this analysis, <strong> the geotechnical and blasting parameters </strong> will be explored to better understand groupings within the data. It is hoped the analysis can be used to eventually optimize blasting to reduce movement and make the fragmentation process more efficient. 

### 1.1) Analysis Method (Selected from Bibliography)

Data clustering refers to the unsupervised classification of features into patterns that then form meaningful groups. Data within 1 cluster are more similar than data in another cluster. As described by Jain et. al [8] there are four main components to a clustering task: 

* Pattern representation - Comprised of selection of the features and extraction for the dataset. This step also involves identifying number of classes in the representation.

* Definition of pattern proximity - This is the distance function defined on the pairs of patterns. For example Euclidean distance is a common measure.

* Clustering or grouping - There are many types of clustering techniques.  

* Data abstraction - Once clustering has been done, a person with knowledge of the domain needs to identify what is represented and ensure the clusters are simple and compact for scalability. 

* Assessment of output - Perform some type of validity analysis on the cluster. 

Hudaverdi [1] uses a hierarchical clustering technique to group 88 blast data into two groups of similarity. Blast design parameters such as burden, spacing, bench height and stemming as well as geotechnical data are used to to obtain two seperate clusters of data. Following this, each cluster is fed into a regression model to predict ground vibration, as indicated by a measure of the Peak Particle Velocity (PPV). A hierarchical technique is used to cluster all blasts into one of two groups. After standardization of the data, the Pearson coefficient of similarity is used to group data. 

### 1.3) Application to This Dataset
Due to the constraint of using the MlLib toolkit - the clustering algorithm that will be explored in this analysis is the K-Means method. K-means is one of the most commonly used clustering algorithms that clusters the data points into a predefined number of clusters. The number of clusters is optimized by reducing the error on the within set sum of squared error. 

The items to be clustered are <strong> individual blasts </strong> based on the features to be described later on this notebook. The blast data  was recorded with instrumentation and manually and provided by Professor Kamran Esmaeili for this investigation.

<a id="plan"></a>
## 2.0) Project Plan & Task Breakdown

<br/>
### 2.1) Project Plan

Based on the CRISP-DM method, these are the proposed tasks for the project:

* <strong>Week 1 - Data Understanding</strong>
  * Data Review 
  * Data verification (with mining professors)
  * Exploratory analysis
  * Review + identify potential analytics techniques
 
* <strong>Week 2 - Data Preparation</strong>
  * Data Cleaning
  * Feature extraction and selection
  * Integrate and format data for model
  * Finalize modeling techniques to be used
  
* <strong>Week 3 - Modeling</strong>
  * Implement selected modeling technique
  * Build model
  * Assess model by computing error
  
* <strong>Week 4 - Evaluation of Alternatives</strong>
  * Evaluate alternative approaches to clustering
  * Review process and next steps
  

<br/>
### 2.2) Task Breakdown
It is anticipated that 10 hours will be allocated each week to perform the tasks required for completion of the project.
<br/>
#### 2.2.1) Data Understanding (Week 1)
The data understanding task involves the following steps: 


* Data Review  (1 hour)
  * Review all tables to develop understanding of data 

* Data verification (3 hours)
  * Meet with Mining Professors to better understand data
  * Eliminate data not required or redundant 
  * Clarify required analyses 

* Exploratory analysis
  * Completed by J.Ponn for select tables

* Project planning (3 hours)
  * Setup notebook
  * Identify breakdown of tasks for each iteration
  * Allocate time estimates for tasks

* Revisions (1 hour)
  * Group discussions about project plans
  * Revisions based on feedback and discussion decisions

* Misc (2 hours)
  * Buffer for unexpected problems, or changes based on group discussion for this iteration
  
#### 2.2.2) Data Preparation (Week 2)
The data preparation task involves the following steps: 


* Data Cleaning (3 hours)
  * Identify tables that need to be cleaned : the important tables are smry_pen_rate, smry_geotech, smry_movement, smry_bmm
  * Normalize the text, remove duplicate or error entries
* Feature extraction and pattern representation (3 hours)
  * Identify features to be used for input to the model.
  * Pattern representation consists of defining the number of classes and features to be used in the clustering algorithm.

* Finalize modeling techniques to be used (2 hours)
  * Model to be considered is the <strong> K-Means</strong>.  
  * The spark.mllib implementation includes a parallelized variant of the k-means++ method called kmeans||.   
* Misc. (2 hour)
  * Group discussion, professor feedback, corrections to notebook 

#### 2.2.3) Modeling (Week 3)
The data modeling task involves the following steps: 

* Integrate and format data for model (2 hours)
  * Identify the steps needed to input the data into the model. 
  * Modify features to be used to ensure they can be passed to the model.
* Build model (5 hours)
  * Run the model, identify clusters
* Assess model (2 hours)
  * Perform validity analysis on the clusters produced
  * Ensure that clusters are tightly bound
  * Compute Within Set of Squared Error (WSSSE)
     

#### 2.2.3) Evaluation of Alternatives (Week 4)
The evaluation task involves the following steps: 

* Evaluate results (6 hours)
  * Run several iterations on the k-means model, with varying K. Use 'Elbow Method' to identify optimal K.
  * Perform other iterations of clustering algorithm.
* Review process and next steps (4 hours)
  * Make changes to the model as necessary. 
  * Draft conclusions and recommendations for next steps
* Misc.
  * Group discussion, professor feedback, corrections to notebook.

<a id="understanding"></a>
## 3.0) Data Understanding

### 3.1) Data Loading

Upload data into databricks, set implicits and define functions to be used to ensure data will be in the type required for processing.

In [8]:
# Import necessary libraries
import pandas as pd # For csv I/O and data formatting
import re # To search through strings
import numpy as np

In [9]:
%scala
import java.sql.Timestamp
import org.apache.commons.io.IOUtils
import java.net.URL
import java.nio.charset.Charset
// val sqlContext= new org.apache.spark.sql.SQLContext(sc)
//      import sqlContext.implicits._


// patching the String class with new functions that have a defualt value if conversion to another type fails.
implicit class StringConversion(val s: String) {
def toTypeOrElse[T](convert: String=>T, defaultVal: T) = try {
    convert(s)
  } catch {
    case _: Throwable => defaultVal
  }
  
  def toIntOrElse(defaultVal: Int = 0) = toTypeOrElse[Int](_.toInt, defaultVal)
  def toDoubleOrElse(defaultVal: Double = 0D) = toTypeOrElse[Double](_.toDouble, defaultVal)
  def toDateOrElse(defaultVal: java.sql.Timestamp = java.sql.Timestamp.valueOf("1970-01-01 00:00:00")) = toTypeOrElse[java.sql.Timestamp](java.sql.Timestamp.valueOf(_), defaultVal)
}

//Fix the date format in this dataset
def fixDateFormat(orig: String): String = {
    val splited_date = orig.split(" ")
    val fixed_date_parts = splited_date(0).split("-").map(part => if (part.size == 1) "0" + part else part)
    val fixed_date = List(fixed_date_parts(0), fixed_date_parts(1), fixed_date_parts(2)).mkString("-")
    val fixed_time = splited_date(1).split(":").map(part => if (part.size == 1) "0" + part else part).mkString(":")
    fixed_date + " " + fixed_time + ":00"
}

In [10]:
%scala
val bmmRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFQnhJY0pfclBLUmM"), Charset.forName("utf8")).split("\n"))

val geotechRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFTjN5WmQ3MmsyTEE"), Charset.forName("utf8")).split("\n"))

val load_timeRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFdS0yU29YdkhiWjQ"), Charset.forName("utf8")).split("\n"))

val movementRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFMUhEMjJoNk1JZTg"), Charset.forName("utf8")).split("\n"))

val truckRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFSkRhc2ZrZDM2aUk"), Charset.forName("utf8")).split("\n"))

val pen_rate_summRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=0BzQSUz4yGQIFS3J3WFJPV21ucW8"), Charset.forName("utf8")).split("\n"))


In [11]:
%scala
case class geotech(             
    Level: Int,
    Blast_Number: String,                                     
    gd_1: String,
    gd_2: String,
    rqd: Double,
    fracs: Double,
    is_50: Double,
    Q: Double
  )

def getgeotechCleaned(row:Array[String]):geotech = {
  return geotech(
    row(0).toIntOrElse(),
    row(1),
    row(2),
    row(3),
    row(4).toDoubleOrElse(),
    row(5).toDoubleOrElse(),
    row(6).toDoubleOrElse(),
    row(7).toDoubleOrElse()
  )
}

val geotech_data = geotechRDD.map(line => line.split(",").map(elem => elem.trim))
val data = geotech_data.filter(s => s(0) != "Level").map(s => getgeotechCleaned(s)).toDF()
data.createOrReplaceTempView("smry_geotech")

In [12]:
%sql -- View the Geotech Summary Table
SELECT *
FROM smry_geotech
LIMIT 10

In [13]:
%scala
case class load_time(               
    BLAST_LOC: String,
    LVL: Int,
    BLAST_NO: Int,
    HS_AVG: Double,
    HS_QT_90: Double,
    HS_QT_10: Double,
    HS_MED: Double,
    HS_NUM_TRUCKS: Double,
    RS_AVG: Double,
    RS_QT_90: Double,
    RS_QT_10: Double,
    RS_MED: Double,
    RS_NUM_TRUCKS: Double
  )
  
def getload_timeCleaned(row:Array[String]):load_time = {
return load_time(
    row(0),
    row(1).toIntOrElse(),
    row(2).toIntOrElse(),
    row(3).toDoubleOrElse(),
    row(4).toDoubleOrElse(),
    row(5).toDoubleOrElse(),
    row(6).toDoubleOrElse(),
    row(7).toDoubleOrElse(),
    row(8).toDoubleOrElse(),
    row(9).toDoubleOrElse(),
    row(10).toDoubleOrElse(),
    row(11).toDoubleOrElse(),
    row(12).toDoubleOrElse()
  )
}


val loading_time = load_timeRDD.map(line => line.split(",").map(elem => elem.trim))
val data = loading_time.filter(s => s(0) != "BLAST_LOC").map(s => getload_timeCleaned(s)).toDF()
data.createOrReplaceTempView("smry_loading_time")

In [14]:
%sql -- View the Loading Time Summary Table
SELECT *
FROM smry_loading_time
LIMIT 10

In [15]:
%scala
case class movement(                 
   BLAST_LOC: String,
   LOC: String,
   BENCH_HT: Int,
   HOLE_DIAM:Int,
   BURDEN: Int,
   SPACING: Int,
   HOLE_CONFIG: String,
   CONFINEMENT: String,
   PATTERN: String,
   STEMMING: Double,
   POWDER_FAC: Double,
   IR_TIME: Int,
   IH_TIME: Int,
   INITIATION_TYPE: String,
   EXPLOSIVE_TYPE: String,
   ROCK_TYPE: String,
   BMM: Int,
   COLLAR_NORTH: Double,
   COLLAR_EAST: Double,
   COLLAR_RL: Double,
   PRE_BMM_RL: Double,
   INST_DEPTH: Double,
   AFTER_EAST: Double,
   AFTER_NORTH: Double,
   SURFACE_RL: Double,
   POST_BMM_RL: Double,
   THREED_DIST: Double,
   H_DIST: Double,
   V_DIST: Double,
   DIRECTION: Double,
   INCLINATION: Double,
   HEAVE: Double
  )
  
def getmovementCleaned(row:Array[String]):movement = {
return movement(
    row(0),
    row(1),
    row(2).toIntOrElse(),
    row(3).toIntOrElse(),
    row(4).toIntOrElse(),
    row(5).toIntOrElse(),
    row(6),
    row(7),
    row(8),
    row(9).toDoubleOrElse(),
    row(10).toDoubleOrElse(),
    row(11).toIntOrElse(),
    row(12).toIntOrElse(),
    row(13),
    row(14),
    row(15),
    row(16).toIntOrElse(),
    row(17).toDoubleOrElse(),
    row(18).toDoubleOrElse(),
    row(19).toDoubleOrElse(),
    row(20).toDoubleOrElse(),
    row(21).toDoubleOrElse(),
    row(22).toDoubleOrElse(),
    row(23).toDoubleOrElse(),
    row(24).toDoubleOrElse(),
    row(25).toDoubleOrElse(),
    row(26).toDoubleOrElse(),
    row(27).toDoubleOrElse(),
    row(28).toDoubleOrElse(),
    row(29).toDoubleOrElse(),
    row(30).toDoubleOrElse(),
    row(31).toDoubleOrElse()
  )
}

val smry_movement = movementRDD.map(line => line.split(",").map(elem => elem.trim))
val data = smry_movement.filter(s => s(0) != "BLAST_LOC").map(s => getmovementCleaned(s)).toDF()
data.createOrReplaceTempView("smry_movement")

In [16]:
%sql -- View the Summary Movement Table
SELECT *
FROM smry_movement
LIMIT 10

In [17]:
%scala
case class bmm(                 
   BLAST_LOC: String,
   GEO_DOM1: String,
   GEO_DOM2: String,
   RQD: Double,
   FRACS_FREQ: Double,
   IS_50: Double,
   Q: Double,
   LEVEL: String,
   PEN_RATE: Double,
   BLAST_DATE: String,
   ROCK_TYPE: String,
   MATERIAL: String,
   HG_MG: Int,
   LG: Int,
   NAG: Int,
   TOTAL_TONS: Int,
   BLAST_TYPE: String ,  
   DH_DELAY: Int,
   HS_DELAY: Int,
   RS_DELAY: Int,
   HOLE_DIA:Int, 
   BENCH_HEIGHT: Int,
   BURDEN: Int,
   SPACING: Int,
   STEMMING: Double,
   SUB_DRILL: Double ,
   EXPLOSIVE: String,
   BLAST_PATTERN: String,
   TOTAL_EXPLOSIVE: Double,
   HOLE_DEPTH: Double,
   FIRING_DIRECTION: String,
   POWDER_FACTOR: Double,
   INITIATION_TYPE: String,
   CONFINEMENT: String,
   BMM_NUM: Int,
   THREED_DIST: Double,
   H_DIST: Double,
   V_DIST: Double,
   DIRECTION: Double,
   INCLINATION: Double,
   HEAVE: Double 
  ) 
   
  
def getbmmcleaned(row:Array[String]):bmm = {
return new bmm(
    row(0),
    row(1),
    row(2),
    row(3).toDoubleOrElse(),
    row(4).toDoubleOrElse(),
    row(5).toDoubleOrElse(),
    row(6).toDoubleOrElse(),
    row(7),
    row(8).toDoubleOrElse(),
    row(9),
    row(10),
    row(11),
    row(12).toIntOrElse(),
    row(13).toIntOrElse(),
    row(14).toIntOrElse(),
    row(15).toIntOrElse(),
    row(16) ,
    row(17).toIntOrElse(),
    row(18).toIntOrElse(),
    row(19).toIntOrElse(),
    row(20).toIntOrElse(),
    row(21).toIntOrElse(),
    row(22).toIntOrElse(),
    row(23).toIntOrElse(),
    row(24).toDoubleOrElse(),
    row(25).toDoubleOrElse(),
    row(26),
    row(27),
    row(28).toDoubleOrElse(),
    row(29).toDoubleOrElse(),
    row(30),
    row(31).toDoubleOrElse(),
    row(32),
    row(33),
    row(34).toIntOrElse(),
    row(35).toDoubleOrElse(),
    row(36).toDoubleOrElse(),
    row(37).toDoubleOrElse(),
    row(38).toDoubleOrElse(),
    row(39).toDoubleOrElse(),
    row(40).toDoubleOrElse() 
  )
}

val smry_bmm = bmmRDD.map(line => line.split(",").map(elem => elem.trim))
val data = smry_bmm.filter(s => s(0) != "BLAST_LOC").map(s => getbmmcleaned(s)).toDF()
data.createOrReplaceTempView("smry_bmm")

In [18]:
%sql -- View the Summary BMM Table
SELECT *
FROM smry_bmm
LIMIT 10

In [19]:
%scala
case class truck(               
    BLAST_LOC: String,
    LVL: Int,
    BLAST_NO: Int,
    HS_AVG: Double,
    HS_QT_90: Double,
    HS_QT_10: Double,
    HS_MED: Double,
    HS_NUM_TRUCKS: Int,
    RS_AVG: Double,
    RS_QT_90: Double,
    RS_QT_10: Double,
    RS_MED: Double,
    RS_NUM_TRUCKS: Int
  )
  
def gettruckCleaned(row:Array[String]):truck = {
return truck(
    row(0),
    row(1).toIntOrElse(),
    row(2).toIntOrElse(),
    row(3).toDoubleOrElse(),
    row(4).toDoubleOrElse(),
    row(5).toDoubleOrElse(),
    row(6).toDoubleOrElse(),
    row(7).toIntOrElse(),
    row(8).toDoubleOrElse(),
    row(9).toDoubleOrElse(),
    row(10).toDoubleOrElse(),
    row(11).toDoubleOrElse(),
    row(12).toIntOrElse()
  )
}

val smry_truck_quantity = truckRDD.map(line => line.split(",").map(elem => elem.trim))
val data = smry_truck_quantity.filter(s => s(0) != "BLAST_LOC").map(s => gettruckCleaned(s)).toDF()
data.createOrReplaceTempView("smry_truck_quantity")

In [20]:
%sql -- View the Summary Truck Quantity Table
SELECT *
FROM smry_truck_quantity
LIMIT 10

In [21]:
%scala
case class penetration_rate_summ(                    
    BLAST_LOC: String,
    LVL: String,
    BLAST_NO: String,
    X: String,                            
    Y: String,                                  
    Z: String,                                   
    PEN_RATE: Double,
    ROCK_TYPE: String
  )
  
def getpenetration_rate_summCleaned(row:Array[String]):penetration_rate_summ = {
return penetration_rate_summ(
    row(0),
    row(1),
    row(2),
    row(3),
    row(4),
    row(5),
    row(6).toDoubleOrElse(),
    row(7)
  )
}

val pen_rates_summ = pen_rate_summRDD.map(line => line.split(",").map(elem => elem.trim))
val data = pen_rates_summ.filter(s => s(0) != "BLAST_LOC").map(s => getpenetration_rate_summCleaned(s)).toDF()
data.createOrReplaceTempView("smry_pen_rate")

In [22]:
%sql -- View the Summary Penetration Rate Table
SELECT *
FROM smry_pen_rate
LIMIT 10

### 3.2) Data Exploration

After loading of the data, conduct data exploration to see what is in the tables.

### Review of Data within the SMRY_GEOTECH Table

In [26]:
%sql
-- How many unique blast levels are there?
SELECT COUNT(LEVEL)
FROM smry_geotech

In [27]:
%sql
-- Counts of distinct levels in smry_geotech
SELECT DISTINCT (LEVEL), COUNT(LEVEL) as counts
FROM smry_geotech
GROUP BY LEVEL 
ORDER BY counts

The levels range from 148 to 208, and include a new row for each new blasting record. So for example, Level 244 has over 120 blast rows associated with it. See the summary table below.

In [29]:
%sql
-- Distinct number of blast numbers in the smry_geotech table
SELECT DISTINCT blast_number 
FROM smry_geotech
GROUP BY blast_number
ORDER BY blast_number DESC

In [30]:
%sql
-- Distribution of RQD values
SELECT rqd
FROM smry_geotech

The largest bin of RQD values is between 90-95 (there are >300) and between 95-100 (almost 450). The values around 0 appear to be errors or blanks and should be removed.

In [32]:
%sql
-- How many distinct rqd values does the smry_geotech have? 
SELECT distinct(rqd)
FROM smry_geotech
GROUP BY rqd

Since there are only 93 distinct rqd values, it appears that these values have been summarized and applied to many more blast locations.

In [34]:
%sql
-- Distribution of fracs
SELECT fracs
FROM smry_geotech

The number of fracs appear to be distributed around 5, with a range from 0 - 14.

In [36]:
%sql
-- Distribution of is_50 values
SELECT is_50
FROM smry_geotech

Distribution of is50 values is around 9, with a range from 4 - 16. The values around 0 appear to be erroneous and should be removed.

In [38]:
%sql
-- Distribution of q values
SELECT q
FROM smry_geotech

Distribution of q values is around 14, with the range mostly between 5 - 25. The values around 0 appear to be erroneous and should be removed.

### Review of Data within the smry_pen_rate table

In [41]:
%sql
-- How many individual blast locaitons are there?
SELECT distinct (BLAST_LOC)
FROM smry_pen_rate

The levels in smry_pen_rate range from 148 - 268, with four records for levels that have alpha-numeric characters. These should be removed.

In [43]:
%sql
-- Counts of distinct levels 
SELECT DISTINCT (lvl), COUNT(lvl) as counts
FROM smry_pen_rate
GROUP BY lvl 
ORDER BY counts

Number of levels range from 160-234 which is similar to that shown in the smry_geotech. There are some discrepancies, such as the alphabetic characters. These should be removed.

In [45]:
%sql
-- Counts of distinct levels 
SELECT DISTINCT (BLAST_NO), COUNT(BLAST_NO) as counts
FROM smry_pen_rate
GROUP BY BLAST_NO 
ORDER BY counts

This table should correspond to the blast_number summary from smry_geotech above. However it's clear that there are large differences. For example, in this table there are many values with alphabetic characters. Ideally they should have a three digit numeric character. These will need to be normalized or removed during cleaning.

In [47]:
%sql
-- Distribution of pen_rate values
SELECT pen_rate
FROM smry_pen_rate

There are many penetrations rates above 2. It is unclear if these are erroneous and should be exploded. Check with the mining professor.

In [49]:
%sql
-- Rock type classifications
SELECT DISTINCT rock_type, count(rock_type)
FROM smry_pen_rate
GROUP BY rock_type

The majority of rock type classification is mf, followed by kp and ovb.

### Review of Data within the smry_bmm table

Following discussions with Prof. Esmaeili and B.Ohadi, another data table was provided (smry_bmm). This table contains geotechnical data, blast parameters and blast outcomes for distinct blast locations. The data was compiled by B.Ohadi.

In [53]:
%sql -- Let's see what's in the smry_bmm table
SELECT *
FROM smry_bmm
LIMIT 10

In [54]:
%sql
-- See how many blast locations per blast level
SELECT LEVEL, COUNT(LEVEL)
FROM smry_bmm
GROUP BY LEVEL
SORT BY COUNT(LEVEL) DESC

Some levels have a large number of blasts - for example, 208 and 256, while some levels have far fewer blast data associated.

In [56]:
%sql
-- See distribution of RQD
SELECT RQD
FROM smry_bmm

Based on visual assessment of the RQD, it appears that the data has been taken from the smry_geotech table. As that table has already been explored above, the subsequent visualizations will focus on the non-geotech parameters.

In [58]:
%sql
-- Box plot of Total Tons distribution
SELECT TOTAL_TONS
FROM smry_bmm

Median tonnage is around 320,00o T, with 75% of data occuring within 200,000 to 600,000 T.

In [60]:
%sql --See distribution of IS_50 values
SELECT is_50
FROM smry_bmm

In [61]:
%sql
-- What Blast Types are there?
SELECT DISTINCT Blast_Type, COUNT(*)
FROM smry_bmm
GROUP BY BLAST_TYPE

Majority of blast types are for production

In [63]:
%sql
-- What is the most prominant HS_delay used in the blasting process?
SELECT DISTINCT HS_DELAY, COUNT(*)
FROM smry_bmm
GROUP BY HS_DELAY

A blasting delay of 42 ms was most used.

In [65]:
%sql
-- What is the most prominant RS_delay used in the blasting process?
SELECT DISTINCT RS_DELAY, COUNT(*)
FROM smry_bmm
GROUP BY RS_DELAY

The most common blasting dleay was 100 ms.

In [67]:
%sql
-- Lets visualize the hole diameter distributions
SELECT DISTINCT HOLE_DIA, COUNT(*)
FROM smry_bmm
GROUP BY HOLE_DIA

The greatest proportion of hole diameter was 216 mm.

In [69]:
%sql
-- What is the most common burden? 
SELECT DISTINCT BURDEN, COUNT(*)
FROM smry_bmm
GROUP BY BURDEN

Burden of 6 m was used in almost all cases. Values of 0 are likely erroneous and should be removed.

In [71]:
%sql
-- What is the most common bencht height? 
SELECT DISTINCT BENCH_HEIGHT, COUNT(*)
FROM smry_bmm
GROUP BY BENCH_HEIGHT

Almost all bench heights were 12 m.

In [73]:
%sql
-- What is the most common spacing? 
SELECT DISTINCT SPACING, COUNT(*)
FROM smry_bmm
GROUP BY SPACING

Most common stemming height was 7 m.

In [75]:
%sql
-- What is the most common stemming? 
SELECT DISTINCT STEMMING, COUNT(*)
FROM smry_bmm
GROUP BY STEMMING

Most common stemming was 4.5 m, followed by 5 m and 4 m.

In [77]:
%sql
-- What is the most common explosive? 
SELECT DISTINCT EXPLOSIVE, COUNT(*)
FROM smry_bmm
GROUP BY EXPLOSIVE

Almost all explosive used was th Fortis Extra 1.05

In [79]:
%sql
-- What types of blast patterns were used? 
SELECT DISTINCT BLAST_PATTERN, COUNT(*)
FROM smry_bmm
GROUP BY BLAST_PATTERN

Almost all th blast patterns were staggered.

In [81]:
%sql
-- What is the most common hole depth? 
SELECT DISTINCT HOLE_DEPTH, COUNT(*)
FROM smry_bmm
GROUP BY HOLE_DEPTH

Hole depth ranged from 1 to 8 m, with most holes drilled to 7 m depth.

In [83]:
%sql
-- What is breakdown of blast initiation types? 
SELECT DISTINCT INITIATION_TYPE, COUNT(*)
FROM smry_bmm
GROUP BY INITIATION_TYPE

Most initiation types were "V" followed by "Echelon"

In [85]:
%sql
-- What is breakdown of confinement? 
SELECT DISTINCT CONFINEMENT, COUNT(*)
FROM smry_bmm
GROUP BY CONFINEMENT

Two types of confinement are presented. Free face and Choked. The values need to be normalized.

In [87]:
%sql
-- What is distribution of three d distances? 
SELECT THREED_DIST
FROM smry_bmm

Most 3d-distance values range between 3.6 to 5.1 m.

In [89]:
%sql
-- What is distribution of horizontal distances? 
SELECT H_DIST
FROM smry_bmm

Similarly, most horizontal distances range from 3.1 m to 4.1 m

In [91]:
%sql
-- What is distribution of vertical distances? 
SELECT V_DIST
FROM smry_bmm

Vertical distance has a greater fluctuation, and a median around 1.5 m.

In [93]:
%sql
-- What is distribution of heave? 
SELECT heave
FROM smry_bmm

Most heave was measured between 2 and 5 m.

In [95]:
%sql
SELECT COUNT(BLAST_LOC)
FROM smry_bmm

### Data Selection
Based on a review of the tables, the smry_bmm table was selected for further analysis. This table was selected as it contains already contains the necessary elements to be assessed with the clustering technique. As it was shown, the Table contains:
* Geotechnical data (Such as RQD, FRAQS_FREQ, Q, and PEN_RATE)
* Blast data (Such as BLAST_LOC, LEVEL)
* Rock data (Such as ROCK_TYPE, MATERIAL)
* Blast parameters (Such as BENCH_HEIGHT, SPACING, STEMMING, SUB_DRILL, POWDER_FAC, INITIATION)
* Blast outcomes (Such as D_Dist, V_Dist, HEAVE )

<a id="preparation"></a>
## 4.0) Data Preparation

### 4.1) Data Cleaning

####List of Cleaning to be Performed 
* Remove GEO_DOM, GEO_DOM1 - These columns appear to not provide any further useful information 
* Remove blast locations where no RQD, FRAQS_FREQ, IS_50 or Q exist, or are 0 - Geotech parameters are an important component of the clustering process
* Remove LEVEL column - not required for analysis 
* Remove BLAST_DATE column - not required for analysis 
* Normalize BLAST PATTERN & CONFINEMENT labels

In [100]:
#Convert spark df to pandas, do not select GEO_DOM, GEO_DOM1, BLAST_DATE, LEVEL, BMM_NUM 
pDf = sqlContext.sql("SELECT BLAST_LOC, RQD, FRACS_FREQ, IS_50, Q, PEN_RATE, ROCK_TYPE, BLAST_TYPE, DH_DELAY, HS_DELAY, RS_DELAY, HOLE_DIA, BENCH_HEIGHT, BURDEN, SPACING, STEMMING, SUB_DRILL, EXPLOSIVE, BLAST_PATTERN, TOTAL_EXPLOSIVE, HOLE_DEPTH, POWDER_FACTOR, INITIATION_TYPE, CONFINEMENT, THREED_DIST, H_DIST,V_DIST, DIRECTION, INCLINATION, HEAVE FROM smry_bmm ").toPandas()

In [101]:
#Confirm that appropriate columns selected
pDf.head()

In [102]:
#Lets whats in the dataframe
pDf.info()

In [103]:
#Remove the rows where RQD, FRACS_FREQ, IS_50, Q and PEN_RATE are zeros
pDf = pDf[(pDf.RQD > 0) & (pDf.FRACS_FREQ > 0) & (pDf.IS_50 > 0) & (pDf.Q > 0) & (pDf.PEN_RATE > 0)]
#We only have 258 rows in the dataframe
pDf.shape

In [104]:
# Normalize string labels by removing excessive whitespace in the middle of a word, shifting to lowercase, and removing any whitespace after the text
def normalizeLabels(label):
  return re.sub('[ ]+',' ',label).lower().rstrip().lstrip()

In [105]:
#Make the labels lowercase and strip blank spaces
pDf['CONFINEMENT'] = pDf['CONFINEMENT'].apply(normalizeLabels)
pDf['BLAST_TYPE'] = pDf['BLAST_TYPE'].apply(normalizeLabels)
pDf['ROCK_TYPE'] = pDf['ROCK_TYPE'].apply(normalizeLabels)
pDf['EXPLOSIVE'] = pDf['EXPLOSIVE'].apply(normalizeLabels)
pDf['BLAST_PATTERN'] = pDf['BLAST_PATTERN'].apply(normalizeLabels)
pDf['INITIATION_TYPE'] = pDf['INITIATION_TYPE'].apply(normalizeLabels)

In [106]:
#Confirm that the labels are normalized
pDf.head()

<a id="modeling"></a>
## 5.0) Modeling

### 5.1) Feature Extraction

According to Hudaverdi [4] the ratios to be used for clustering are :
* Spacing (S) / Burden (B)
* Bench Height (H) / Burden (B)
* Burden (B) / Hole Diameter (D)
* Stemming (T) / Burden (B)
* Subdrill (U) / Burden (B)
* Powder Factor

In [110]:
%sql
SELECT *
FROM smry_bmm
LIMIT 5

In [111]:
#Calculate ratios
pDf['SB_ratio'] = pDf['SPACING'] / pDf['BURDEN']
pDf['HB_ratio'] = pDf['BENCH_HEIGHT'] / pDf['BURDEN']
pDf['BD_ratio'] = pDf['BURDEN'] / pDf['HOLE_DIA']
pDf['TB_ratio'] = pDf['STEMMING'] / pDf['BURDEN']
pDf['UB_ratio'] = pDf['SUB_DRILL'] / pDf['BURDEN']

In [112]:
#Inspect dataframe to ensure the ratios have been calculated
pd.set_option('display.max_columns', None)
pDf.head()

In [113]:
# Convert a label feature into an index number
def facorizeFeature(feat, df, labelDict):
  # Convert string labels into an index number
  labels, levels = pd.factorize(df[feat])
  
  # Add index numbers to main dataframe
  df[feat] = labels
  
  # Store labels in dictionary for later reference
  labelDict[feat] = levels
  
  return df, labelDict

In [114]:
# Initalize label dictionary
mLabelDict = {}

In [115]:
# Apply label normalization to data
pDf, mLabelDict = facorizeFeature('ROCK_TYPE', pDf, mLabelDict)
pDf, mLabelDict = facorizeFeature('BLAST_TYPE', pDf, mLabelDict)
pDf, mLabelDict = facorizeFeature('EXPLOSIVE', pDf, mLabelDict)
pDf, mLabelDict = facorizeFeature('BLAST_PATTERN', pDf, mLabelDict)
pDf, mLabelDict = facorizeFeature('INITIATION_TYPE', pDf, mLabelDict)
pDf, mLabelDict = facorizeFeature('CONFINEMENT', pDf, mLabelDict)

In [116]:
#Check the dataframe to ensure strings have been factorized
pDf.head()

In [117]:
#See the indexed labels
mLabelDict

In [118]:
#Drop columns that are not required for the clustering process
features = pDf.drop(['BLAST_LOC','DIRECTION','INCLINATION','HEAVE'],1)

In [119]:
#Drop columns that include blast movement as these are outcomes 
features.drop(['THREED_DIST','H_DIST', 'V_DIST'],1,inplace='true')

In [120]:
#Drop columns that created ratios for
features.drop(['HOLE_DIA','BENCH_HEIGHT','BURDEN','SPACING','STEMMING','SUB_DRILL' ],1,inplace='true')

In [121]:
#View the dataframe to ensure the correct columns are in place
features.tail()

In [122]:
#Count the number of times that infinity occurs in the dataset
np.isinf(features).sum()

In [123]:
#Check if the dataframe has any null values 
features.isnull().sum()

In [124]:
#Convert infinities to NaN's so they can all be handled together
features.replace(np.inf, np.nan,inplace='True')

In [125]:
#Confirm infinities are removed
np.isinf(features).sum()

In [126]:
#Remove the NaN rows from the dataframe
features.dropna(inplace='True')

In [127]:
#Check the shape of the features dataframe now that the NaN's have been dropped
features.shape

In [128]:
#Create a list of the features 
featuresList = list(features)

In [129]:
#Let's see the features list to ensure it has all the features we want
featuresList

### 5.2) Train the Model - Base Case

The first iteration of the model involves trialling with all the initial set of data and with a randomly selected k. The results from this assessment are not representative of the true clustering.

The parameters to be used are those as default in the spark.mllib implementation guide. Max iterations have been increased to 100, and the initialization mode is kept as random.

In [132]:
#Import necessary libraries 
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt
from pyspark.mllib.stat import Statistics
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
from sklearn import cluster
from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

In [133]:
#Parse the features pandas dataframe into an RDD of features which will be input into the model
featuresRDD = sqlContext.createDataFrame(features).rdd.map(lambda row: np.array([float(item) for item in row]))

In [134]:
#Let's see some summary statistics of the Features RDD
summary = Statistics.colStats(featuresRDD)
print(summary.mean())  # a dense vector containing the mean value for each column
print(summary.variance())  # column-wise variance
print(summary.numNonzeros())  # number of nonzeros in each column

As it can be seen - several columns have a variance of 0, indicating that these columns will likely not add substantially to the clustering. The columns are columns 8 - 12. In later iterations, these columns will be removed.

In [136]:
#Train the clustering algorithm using the Features RDD. Try with 4 clusters in the first iteration.
clusters = KMeans.train(featuresRDD, 4, maxIterations=100, initializationMode="random")

In [137]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [138]:
#Computer the error 
WSSSE = featuresRDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

In [139]:
#Let's look at the coordinates of the centre points of the clusters, to help understand which features are the most discriminatory. 
d = clusters.clusterCenters
clusterDf = pd.DataFrame(d)
#Pass the feautures list as column labels 
clusterDf.columns = featuresList

In [140]:
#View the cluster centre coordinates for all four clusters
clusterDf

In [141]:
#View a stastical summary of the cluster centre coordinates
clusterDf.describe()

Without normalization of the data, it's not very useful to describe mean and distribution of the data. However, based on an visual inspection of the features, the following features appear to be the most important in determining clusters, based on the differences between their cluster coordinates:
* FRACS_FREQ
* IS_50
* Q
* PEN_RATE

Most of the blast parameters have very similar cluster centre coordinates. This indicates that the blast parameters are not very useful in helping to determine cluster assignments.

<a id="evaluation"></a>
## 6.0) Alternative Iterations of Clustering

### 6.1) Increasing K Sequentially

In order to evaluate the best number of clusters, the Elbow Method approach is carried out. Starting from k=1, the K-means algorithm is run on the same dataset with increasing k and observing the value of the cost function WSSE. The number of clusters selected should be 1 greater than the 'elbow' on the graph, indicating the biggest drop of WSSE.

In [146]:
#Initialize the empty array
cluster_elbow = []
errorGraph = []

In [147]:
#Define the function which will be used to compute the cost for each iteration
def error_iter(clusters_iter, point):
    center = clusters_iter.centers[clusters_iter.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [148]:
#define function to compute WSSE as K increases sequentially 
def num_k_iter(num_k, fRDD):
    for k in range(1,num_k):
      #Train the model, with k increasing from 1 to entered value
      clusters_iter = KMeans.train(fRDD, k, maxIterations=100, initializationMode="random")
      #Calculate the WSSE errors for each k
      WSSE_iter = fRDD.map(lambda point: error_iter(clusters_iter, point)).reduce(lambda x, y: x + y) 
      #Append it to an initialized list   
      errorGraph.append([k,WSSE_iter])   

In [149]:
#Let's try out the funcion with k from 1 to 10
num_k_iter(10,featuresRDD)

In [150]:
#Let's see what's in the Cluster Elbow array
cluster_elbow = errorGraph

In [151]:
#Create a dataframe so we can viusalize the Cluster Elbow graph
cluster_df = sqlContext.createDataFrame(cluster_elbow)
cluster_df.createOrReplaceTempView('elbows')

In [152]:
%sql -- Plot the number of K vs. the WSSE
SELECT _1 as NumK, _2 as WSSSE
FROM elbows

The biggest drop occurs after the 2'nd cluster so this analysis suggests that 3 clusters would be the optimal solution for this dataset.

### 6.2) With Normalized Columns in Features Dataframe

Previously it was noted that features have large differences in actual values. For example RQD values are mostly in the range from 80 - 100, while FRACS_FREQ are largely between 2 - 6.These differences may cause discrepancies in the clustering process and should therefore be normalized. 

The method used by Hudaverdi [1] to normalize data is the Z-score normalization technique. In this method, each value in the feature table is adjusted by subtracting the column mean and dividing by the column standard deviation.

In [156]:
#Create a function that will normalize each value in the dataframe. Normalization involves subtracting each value by the column mean and dividing by the column standard deviation. 
def normalize(df):
    features_normalized = df.copy()
    for feature_name in df.columns:
        col_mean = df[feature_name].mean()
        col_std = df[feature_name].std()
        features_normalized[feature_name] = (df[feature_name] - col_mean) / col_std
    return features_normalized

In [157]:
#Let's see the original features table prior to normalization
features.head()

In [158]:
#Pass the features to be normalized to the function
normalized_features = normalize(features)
#See the new normalized values 
normalized_features.head()

After normalization, a few columns appear to produce a lot of NaN values. This occus because all the values in these colums are the same, therefore subtracting the mean results in 0. Let's see how many are NaNs.

In [160]:
#View the null values in the normalized_features dataframe
normalized_features.isnull().sum()

In [161]:
#Remove NaN columns from normalized_feautres
normalized_features.drop(['DH_DELAY','HS_DELAY','RS_DELAY','EXPLOSIVE','BLAST_PATTERN'],1,inplace='true')

In [162]:
#Create a list of the column headers 
normalized_feautresList = list(normalized_features)
#Let's view the dataframe
normalized_features.head()

In [163]:
#Parse the features pandas dataframe into an RDD of features which will be input into the model
normalized_featuresRDD = sqlContext.createDataFrame(normalized_features).rdd.map(lambda row: np.array([float(item) for item in row]))

In [164]:
#Run the clustering function 
num_k_iter(10,normalized_featuresRDD)

In [165]:
#Pass the error graph to a new variable
normalizedElbow = errorGraph

In [166]:
#Create a spark dataframe so we can viusalize the Cluster Elbow graph
NormCluster_df = sqlContext.createDataFrame(normalizedElbow)
NormCluster_df.createOrReplaceTempView('norm_elbows')

In [167]:
%sql -- Plot normalized WSSE Error vs. Num K
SELECT _1 as NumK, _2 as WSSSE
FROM norm_elbows

Based on this graph, the node after largest drop is 3, therefore it's recommended that 3 clusters be used for this dataset. Let's conduct some more exploration with 3 clusters and see how the data look.

In [169]:
#Re run the model training with 3 clusters specified
normClusters = KMeans.train(normalized_featuresRDD, 3, maxIterations=100, initializationMode="random")

In [170]:
#Create an array of cluster centers
normCentres = normClusters.clusterCenters
#Pass the array to a pandas dataframe
normCentresDf = pd.DataFrame(normCentres)

In [171]:
#Add column headers to allow for visual assessment of features
normCentresDf.columns = normalized_feautresList
#Let's view the centre coordinates of each normalized table
normCentresDf

The centre coordinates indicate the location of each cluster. Although we don't know the tightness of the cluster, we can use the centre coordinates to visualize the splits among the features.

In [173]:
#Create a spark dataframe so we can viusalize the Cluster Elbow graph
NormClusterCentre_df = sqlContext.createDataFrame(normCentresDf)
NormClusterCentre_df.createOrReplaceTempView('norm_cluster_cent')

In [174]:
%sql -- Let's see all the columns in our centre cluster dataframe
SELECT *
FROM norm_cluster_cent

In [175]:
%sql -- Differences in RQD values among the normalized clusters shows three relatively distinct groupings of clusters 
SELECT RQD
FROM norm_cluster_cent

In [176]:
%sql -- Differences in FRACS_FREQ values among the normalized clusters shows three distinct clusters.
SELECT FRACS_FREQ
FROM norm_cluster_cent

In [177]:
%sql -- Differences in IS_50 values among the normalized clusters shows that clusters 0 and 1 are more similar to each other than to 2
SELECT IS_50
FROM norm_cluster_cent

In [178]:
%sql -- Differences in Q values among the normalized clusters shows that 0,2 are more similar to each other than to 1. 
SELECT Q
FROM norm_cluster_cent

In [179]:
%sql -- Differences in PEN_RATE values among the normalized clusters shows that clusters 0 and 2 are more similar to each other than to 0
SELECT PEN_RATE
FROM norm_cluster_cent

In [180]:
%sql -- Differences in RQD values among the normalized clusters shows that clusters 1 and 2 are more similar to each other than to 0
SELECT ROCK_TYPE
FROM norm_cluster_cent

In [181]:
%sql -- Differences in confinement values among the normalized clusters shows that clusters 0 and 2 are more similar to each other than to 2
SELECT CONFINEMENT
FROM norm_cluster_cent

In [182]:
%sql -- Differences in ROCK_TYPE values among the normalized clusters shows that clusters 1 and 2 are more similar to each other than to 0
SELECT ROCK_TYPE
FROM norm_cluster_cent

In [183]:
%sql -- Differences in SB_Ratio values among the normalized clusters shows that only 1 cluster was formed.
SELECT SB_RATIO
FROM norm_cluster_cent

In [184]:
%sql -- Differences in HB_RATIO values among the normalized clusters shows that only 1 cluster was formed.
SELECT HB_RATIO
FROM norm_cluster_cent

In [185]:
%sql -- Differences in TB_RATIO values among the normalized clusters shows that only 1 cluster was formed. 
SELECT TB_RATIO
FROM norm_cluster_cent

### 6.3) With Just the Geotechnical Parameters

Based on the results of the previous clustering, it appears that many of the blast parameters are not as useful for assigning discriminant clusters. As such, we should try assign clusters with just the geotechnical parameters to see if there are interesting groupings within the data.

In [188]:
#Create a geotech data features list
normalizedGeotech = normalized_features[['RQD','FRACS_FREQ','Q','PEN_RATE','ROCK_TYPE']]

In [189]:
#Let's view the new table
normalizedGeotech.head()

In [190]:
#Ensure it has the correct shape
normalizedGeotech.shape

In [191]:
#Create the RDD
normalized_geofeaturesRDD = sqlContext.createDataFrame(normalizedGeotech).rdd.map(lambda row: np.array([float(item) for item in row]))

In [192]:
#Pass it to the clustering function
num_k_iter(10,normalized_geofeaturesRDD)

In [193]:
#Create an elbow graph with the WSSSE error points
normalized_geoElbow = errorGraph

In [194]:
#Create a spark dataframe so error points can be visualized
NormgeoCluster_df = sqlContext.createDataFrame(normalized_geoElbow)
NormgeoCluster_df.createOrReplaceTempView('norm_geo_elbows')

In [195]:
%sql -- Let's view the WSSE vs. Num K
SELECT _1 as NumK, _2 as WSSSE
FROM norm_geo_elbows

Similar to previous iterations, it appears the best K is 3 indicating that the data can be grouped into three distinct clusters based on the geotechnical features.

## 7.0) Conclusions & Recommendations

Based on analysis conducted, the following observations are made :<br/>
* The K-means algorithm is useful for grouping data based on underlying similarity 
* Based on running the K-means algorithm with sequentially increasing K, the optimal number of clusters was found to be 3. This was also found with just the geotechnical dataset - which indicates validation of the initial test. This suggests that there are 3 underlying groupings of the data.
* Upon reviewing the centroids of the coordinates, it was typically noted that two of the clusters were closer to each other than the third, however this alternated depending on which parameter was assessed. 
* The blast parameters did not provide useful discriminant data to perform the clustering. This was shown by utilizing just the geotechnical data and obtaining 3 clusters - the same result as conducted with blast parameters. This suggests that most blasts have very similar data points, and is supported by the graphical plots depicted in Section 3.


The following recommendations are made regarding data management and future work:
* The dataset used for the final clustering was relatively small - only 217 points. This was as a result of data processing and cleaning which resulted in the removal of many datapoints. In order to build better models, a larger more accurate dataset should be collected. 
* The dataset used for the analysi (smry_bmm) contained many data that had been averaged from one location across many different points. This is understandable considering the limitations of data collections at a mine, however additional efforts should be made to collect discrete geospatial data which are representative - for example the RQD, Q, IS_50 attached to individual blast hole data points. 
* This analysis provided distinct clusters into which the data can be grouped based on similarity. Further work can develop regression models to predict movement or fragmentation based on the clusters. These will likely result in better predictive models. 
* Documentation for ML Lib is still being developed (or difficult to find) as such, detailed analysis on clusters was not possible. For example it would have been useful to see cluster assignments and relate it back to individual data points.

<a id="bibliography"></a>
## 7.0) Bibliography
<strong>Selected Paper:</strong><br/>
<a id="ref1"></a>
[1] T. Hudaverdi. Application of Multivariate Analysis for Prediction of Blast Induced Ground Vibrations. In Soil Dynamics and Earthquake Engineering. Pages 300 – 308, 2012.<br/><br/>
<a id="ref2"></a>

<strong>Also cited in literature review:</strong><br/>
[2] M. Rezaei, M. Monjezi, A. Varjani. Development of a fuzzy model to predict flyrock in surface mining. In Safety Science, pages 298-305, SS49, 2011<br/><br/>
<a id="ref3"></a>
[3] R. Trivedi, T. Singh, N. Gupta. Prediction of Blast-Induced Flyrock in Opencast Mines Using ANN and ANFIS. <br/><br/>
<a id="ref4"></a>
[4] CVB Cunningham. The Kuz-Ram fragmentation model – 20 years on. In Brighton Conference Proceedings, pages 201 – 210, 2005 <br/><br/>
<a id="ref5"></a>
[5] E. Hamdi, J. du Mouza. A methodology for rock mass characterization and classification to improve blast results. International Journal of Rock Mechanics and Mining Sciences. 42 Pages 177 – 194, 2005. <br/><br/>
<a id="ref6"></a>
[6] W. Zhou and N. Maerz. Implementation of multivariate clustering methods for characterizing discontinuities data from scanlines and oriented boreholes. Computers and Geosciences. Vol 28 Pages 827 – 839, 2002.<br/><br/>
<a id="ref7"></a>
[7] A Jain, M Murty, and P Flynn. Data Clustering : A Review. In ACM Digital Library. Vol 31, Iss 3. Pages 264 – 323, 1999.<br/><br/>
<a id="ref8"></a>
[8] A Jain. Data Clustering : 50 Years Beyond K-Means. Published in 19th International conference in Pattern Recognition (ICPR) . Vol 31, Iss. 8, Pages 651 – 666, 2010.<br/><br/>